**Importing the libraries**

In [0]:
import numpy as np 
import pandas as pd 
import os
import gc
import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import cv2
from tqdm import tqdm
# Load the TensorBoard notebook extension
%load_ext tensorboard 
# Clear any logs from previous runs
!rm -rf ./logs/ 
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import datetime

Using TensorFlow backend.


**Loading dataset**

On extraction of the kaggle.json file, we upolad that file to runtime. Also, we need to go to Planet Competition(website) for the dataset and need to accept the rules for the competition. After that I have downloaded alll the dateset related files, and uploaded it to my runtime while execution. It contains the image folder as well. For testing purpose, I have taken few images.

In [0]:
 #! mkdir -p ~/.kaggle/
 #! mv kaggle.json ~/.kaggle/

In [0]:
# path = Config.data_path()/'planet'
# path.mkdir(parents=True, exist_ok=True)
# path

In [0]:
#  ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p {path}  
#  ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv -p {path}  
#  ! unzip -q -n {path}/train_v2.csv.zip -d {path}

In [0]:
x_train = []
x_test = []
y_train = []
df_train = pd.read_csv('train_v2.csv')

In [0]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('train_996.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

print(x_train.shape)
print(y_train.shape)

split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          validation_data=(x_valid, y_valid))
          
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)
print(y_valid)
print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))


100%|██████████| 40479/40479 [00:59<00:00, 685.63it/s]


(40479, 32, 32, 3)
(40479, 17)
Train on 35000 samples, validate on 5479 samples
Epoch 1/4
35000/35000 [==============================] - 118s 3ms/step - loss: 0.2907 - accuracy: 0.8939 - val_loss: 0.2547 - val_accuracy: 0.9056
Epoch 2/4
35000/35000 [==============================] - 118s 3ms/step - loss: 0.2642 - accuracy: 0.9050 - val_loss: 0.2553 - val_accuracy: 0.9056
Epoch 3/4
35000/35000 [==============================] - 117s 3ms/step - loss: 0.2618 - accuracy: 0.9051 - val_loss: 0.2547 - val_accuracy: 0.9056
Epoch 4/4
35000/35000 [==============================] - 118s 3ms/step - loss: 0.2606 - accuracy: 0.9051 - val_loss: 0.2558 - val_accuracy: 0.9056
[[1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
[[0.07963142 0.00379553 0.31782472 ... 0.00358364 0.06675479 0.10895309]
 [0.07963142 0.00379553 0.31782472 ... 0.00358364 0.06675479 0.10895309]
 [0.07963142 0.00379553 0.31782472 ... 0.00358364 0.06675479 0.108

In [0]:
!tensorboard dev upload --logdir ./logs \

2020-05-06 06:53:54.114786: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Data for the "graphs" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
Upload started and will continue reading any new data as it's added
to the logdir. To stop uploading, press Ctrl-C.
View your TensorBoard live at: https://tensorboard.dev/experiment/EnreNEOOTmCEJ80VXdmFpw/
